In [1]:
!pip install keplergl --quiet
import numpy as np
import pandas as pd
import geopandas as gpd
import keplergl
#from keplergl import KeplerGL
import matplotlib as plt
import plotly.express as px
#import h3
import shapely
from geopandas import GeoDataFrame,GeoSeries
import folium
import matplotlib
%matplotlib inline

In [3]:
bikelanes = pd.read_csv('bikelanes_processed.csv')
road_network = pd.read_csv('road_network_processed.csv')
traffic_volumes = pd.read_csv('traffic_volumes_h3.csv')

In [26]:
#merging the two dataframes on the hex_cell column
merge = road_network.merge(bikelanes, on='hex_cell', how='inner')

In [27]:
merge.head()

,Unnamed: 0_x,official_street_name,road_class,geometry,latitude,longitude,coordinates_x,hex_cell,Unnamed: 0_y,installed,street_name,from_street,to_street,infra_loworder,infra_highorder,coordinates_y
0,223,WELLESLEY COTTAGES,Local / Strata,LINESTRING (-79.36711648017713 43.668687188536...,43.668863,-79.367188,"(43.66886268856369, -79.36718833021547)",8a2b9bc718dffff,2278,2005,Wellesley St E,Parliament St,Sumach St,Signed Route (No Pavement Markings),Signed Route (No Pavement Markings),"(43.66876274034707, -79.36679358103952)"
1,56413,WELLESLEY STREET EAST,Local / Street,LINESTRING (-79.36652357999942 43.668817788586...,43.669003,-79.365687,"(43.66900293865787, -79.36568742974895)",8a2b9bc718dffff,2278,2005,Wellesley St E,Parliament St,Sumach St,Signed Route (No Pavement Markings),Signed Route (No Pavement Markings),"(43.66876274034707, -79.36679358103952)"
2,56418,WELLESLEY STREET EAST,Local / Street,LINESTRING (-79.36711648017713 43.668687188536...,43.668752,-79.366820,"(43.668752488561395, -79.36682003008828)",8a2b9bc718dffff,2278,2005,Wellesley St E,Parliament St,Sumach St,Signed Route (No Pavement Markings),Signed Route (No Pavement Markings),"(43.66876274034707, -79.36679358103952)"
3,225082,LANE SOUTH WELLESLEY EAST SACKVILLE STREET,Alleyway / Laneway,LINESTRING (-79.36595597979547 43.668558088567...,43.668604,-79.365733,"(43.66860392343607, -79.36573268672439)",8a2b9bc718dffff,2278,2005,Wellesley St E,Parliament St,Sumach St,Signed Route (No Pavement Markings),Signed Route (No Pavement Markings),"(43.66876274034707, -79.36679358103952)"
4,228927,COLTSFOOT LANE,Alleyway / Laneway,LINESTRING (-79.36640367993246 43.668491588534...,43.668543,-79.366176,"(43.66854287137301, -79.36617636427408)",8a2b9bc718dffff,2278,2005,Wellesley St E,Parliament St,Sumach St,Signed Route (No Pavement Markings),Signed Route (No Pavement Markings),"(43.66876274034707, -79.36679358103952)"


In [28]:
merge.shape

(5836, 16)

In [29]:
traffic_volumes.shape

(2209, 6)

In [30]:
road_network.shape

(50066, 8)

In [ ]:
merge.to_csv('merge.csv', index=False)

Executing shutdown due to inactivity...


2023-08-01 20:27:01,014 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2023-08-01 20:27:01,089 - INFO     - Executing shutdown...
/Users/rakeenahmed/opt/anaconda3/lib/python3.9/site-packages/dtale/app.py:422: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



_________________________________

In [2]:
traffic_volumes = gpd.read_file('traffic_volumes_h3.csv')

In [3]:
traffic_volumes.head()

,hex_cell,cyclists_count,pedestrians_count,total_count,trucks_count,vehicle_count,geometry
0,8a2b986c8477fff,7,2,21055,246,21046,None
1,8a2b986ca71ffff,0,0,8783,149,8783,None
2,8a2b986d9297fff,7,3,24301,655,24291,None
3,8a2b986d96effff,5,1,25822,648,25816,None
4,8a2b986db25ffff,26,2021,61237,1412,59190,None


In [5]:
import h3

In [8]:
h3.cell_to_latlng('8a2b986c8477fff')

(43.85560695874397, -79.16970050609908)

In [9]:
help(h3.cell_to_latlng)

Help on method cell_to_latlng in module h3.api._api_template:

cell_to_latlng(h) method of h3.api._api_template._API_FUNCTIONS instance
    Return the center point of an H3 cell as a lat/lng pair.
    
    Parameters
    ----------
    h : H3Cell
    
    Returns
    -------
    lat : float
        Latitude
    lng : float
        Longitude



In [12]:
traffic_volumes['coordinates'] = traffic_volumes.apply(lambda x: h3.cell_to_latlng(x['hex_cell']), axis=1)

In [14]:
traffic_volumes.head()

,hex_cell,cyclists_count,pedestrians_count,total_count,trucks_count,vehicle_count,geometry,coordinates
0,8a2b986c8477fff,7,2,21055,246,21046,None,"(43.85560695874397, -79.16970050609908)"
1,8a2b986ca71ffff,0,0,8783,149,8783,None,"(43.84877558112413, -79.19953212660344)"
2,8a2b986d9297fff,7,3,24301,655,24291,None,"(43.842534107292124, -79.22649042309308)"
3,8a2b986d96effff,5,1,25822,648,25816,None,"(43.841339873955704, -79.23221827585886)"
4,8a2b986db25ffff,26,2021,61237,1412,59190,None,"(43.83688442358659, -79.25057773031612)"


In [18]:
traffic_volumes.coordinates[0][1]

-79.16970050609908

In [20]:
traffic_volumes['geometry'] = traffic_volumes['coordinates'].apply(lambda x: shapely.geometry.Point(x))

In [22]:
traffic_volumes.head()

,hex_cell,cyclists_count,pedestrians_count,total_count,trucks_count,vehicle_count,geometry,coordinates
0,8a2b986c8477fff,7,2,21055,246,21046,POINT (43.85561 -79.16970),"(43.85560695874397, -79.16970050609908)"
1,8a2b986ca71ffff,0,0,8783,149,8783,POINT (43.84878 -79.19953),"(43.84877558112413, -79.19953212660344)"
2,8a2b986d9297fff,7,3,24301,655,24291,POINT (43.84253 -79.22649),"(43.842534107292124, -79.22649042309308)"
3,8a2b986d96effff,5,1,25822,648,25816,POINT (43.84134 -79.23222),"(43.841339873955704, -79.23221827585886)"
4,8a2b986db25ffff,26,2021,61237,1412,59190,POINT (43.83688 -79.25058),"(43.83688442358659, -79.25057773031612)"


In [28]:
traffic_volumes.to_csv('traffic_volume_processed.csv', index=False)